# InceptionV3

Model to classify the food 101 images, based on the Inception V3
<br>
Tested and working with tensorflow 2.1.0 & keras 2.3.1 
- 67988 training images
- 22716 test images


In [2]:
# Install Keras version 2.3.1
%pip install keras==2.13.1

  Obtaining dependency information for keras==2.13.1 from https://files.pythonhosted.org/packages/2e/f3/19da7511b45e80216cbbd9467137b2d28919c58ba1ccb971435cb631e470/keras-2.13.1-py3-none-any.whl.metadata
Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
##################################
%conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0
# Anything above 2.10 is not supported on the GPU on Windows Native
%python -m pip install "tensorflow=2.10"
# Verify install:
%python -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"

In [11]:
###############################
import tensorflow
from tensorflow import keras


In [3]:
%pip install plot_model

Note: you may need to restart the kernel to use updated packages.


In [12]:
#%tensorflow_version 2.x
import os
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential, Model 
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, MaxPooling1D, GlobalAveragePooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense, Input 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau 
from keras.callbacks import TensorBoard
from keras import backend as K 
from keras.optimizers import Adam, SGD, Adadelta
from keras.regularizers import l2, l1
import cv2
from keras.callbacks import CSVLogger
import sys
import time
import pickle
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
#from keras.utils.vis_utils import plot_model
import plot_model
import numpy as np
import matplotlib.pyplot as plt

from keras.applications.inception_v3 import InceptionV3

In [ ]:
# Downloading zip file containing images
import zipfile
with zipfile.ZipFile('C:\\Users\\chris\\Onedrive\\Desktop\\ds340w\\images.zip', 'r') as zip_ref:
    zip_ref.extractall('C:\\Users\\chris\\Onedrive\\Desktop\\ds340w')

In [4]:
# Loading images
#train_data_dir = '/content/images/train'
#validation_data_dir = '/content/images/test'
nb_train_samples = 67988 
nb_validation_samples = 22716
n_classes = 101
epochs = 10
batch_size = 75

In [5]:
# Checking image format: if RGB channel is coming first or last so, model will check first and then input shape will be feeded accordingly.
img_width = 299
img_height = 299

if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3) 

**Model**
<br>
- InceptionV3 base model, with imagenet weights loaded
- Average Pooling layer with pool size 8x8
- Dropout with probability 0.4
- Flatten layer
- Dense Layer of 101 neurons

In [6]:
#Model that enable the freezing of the resnet layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(299, 299, 3)))
x = base_model.output
x = AveragePooling2D(pool_size=(8, 8))(x)
x = Dropout(.4)(x)
x = Flatten()(x)

predictions = Dense(n_classes,
                    kernel_regularizer=l2(0.005),
                    activity_regularizer=l1(0.005), 
                    activation='softmax')(x)

model = Model(base_model.input, predictions)

In [ ]:
#from tensorflow.keras.utils import plot_model
#plot_model(model, to_file='model_cnn.png')

In [7]:
# Compile the model using Stochastic Gradiend Descent (SGD) optimizer
model.compile(
    optimizer=SGD(learning_rate=.01, momentum=.9), 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

In [ ]:
# Plot the model and save to file
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [8]:
# Train & Test Data Generators with image augmentation 
train_data_dir = '.\\train'
validation_data_dir = '.\\test'

train_datagen = ImageDataGenerator(
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    zoom_range=[.8, 1],
    channel_shift_range=30,
    fill_mode='reflect')

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = 11,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = 11,
    class_mode='categorical')

Found 67988 images belonging to 101 classes.
Found 22716 images belonging to 101 classes.


In [13]:
# Setup callbacks and logs 
checkpoint_path = "InceptionV3/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
cp = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy',save_best_only=True,verbose=1, mode='max')
csv_logger = keras.callbacks.CSVLogger('InceptionV3/InceptionV3.log')

In [14]:
# Reduce LR if no improvement on the test accuracy is observed
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,
                              patience=2, min_lr=0.00001)

In [ ]:
# Loading best model weights
#model.load_weights('')

**Model Training**

In [15]:
#Fitting
model.fit(train_generator,
          steps_per_epoch = nb_train_samples // batch_size,
          validation_data=validation_generator,
          validation_steps=nb_validation_samples // batch_size,
          epochs=2,
          verbose=1,
          callbacks=[csv_logger]
          )

Epoch 1/2
906/906 [==============================] - 20265s 22s/step - loss: 2.9591 - accuracy: 0.4433 - val_loss: 2.8616 - val_accuracy: 0.4008
Epoch 2/2
906/906 [==============================] - 20653s 23s/step - loss: 1.9919 - accuracy: 0.5937 - val_loss: 2.1181 - val_accuracy: 0.5584


In [16]:
# Read log file
#df = pd.read_csv('InceptionV3/InceptionV3.log')
df = pd.read_csv('InceptionV3/InceptionV3.log')

In [17]:
# Training and Test accuracy
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['epoch'], y=df['accuracy'],
                    mode='lines',
                    name='training'))

fig.add_trace(go.Scatter(x=df['epoch'], y=df['val_accuracy'],
                    mode='lines',
                    name='test'))


fig.update_layout(
    font_size = 20,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)

fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='Gray')
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='Gray')

In [18]:
# Training and Test loss
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['epoch'], y=df['loss'],
                    mode='lines',
                    name='training loss'))

fig.add_trace(go.Scatter(x=df['epoch'], y=df['val_loss'],
                    mode='lines',
                    name='test loss'))


fig.update_layout(
    font_size = 20,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)

fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='Gray')
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='Gray')

**Model evaluation**

In [19]:
#Evaluation
score = model.evaluate_generator(
    validation_generator, 
    nb_validation_samples/batch_size)

print(score)

C:\Users\chris\AppData\Local\Temp\ipykernel_12000\798875452.py:2: UserWarning:

`Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.



[2.1186742782592773, 0.5584610104560852]
